In [17]:
import pandas as pd
import numpy as np
import re
from IPython.display import display

## NZZ CS DATA PREP

#### import and merge data

In [18]:
# import nzz cs data

file_paths = [
    "Data/NZZ/nzz_scraped_data_CS_april_part1.csv",
    "Data/NZZ/nzz_scraped_data_CS_april_part2.csv",
    "Data/NZZ/nzz_scraped_data_CS_april_part3.csv",
    "Data/NZZ/nzz_scraped_data_CS_april_part4.csv",
    "Data/NZZ/nzz_scraped_data_CS_april_part5.csv"
]

# List to store DataFrames
dfs = []

# Read each CSV file into a DataFrame and append to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
nzz_data_CS_merged = pd.concat(dfs, ignore_index=True)

In [19]:
len(nzz_data_CS_merged)

1557

In [20]:
nzz_data_CS_merged.head()

,article_title,headline,article_content,text_date
0,Die neue «Savoy Brasserie & Bar»: Prestige un...,Das traditionsreiche Hotel Savoy im Herzen Zür...,Er bildet beim Monopoly nicht zufällig das beg...,18.04.2024
1,Briefing am Mittwochabend,Nationalrat gegen Beitritt zur Oligarchen-Task...,Das Wichtigste am Abend: Die Ablehnung des Bei...,17.04.2024
2,Chefwechsel bei CS-Tochter Bank Now: Nun sieh...,Die Konsumkreditbank gehört seit der CS-Überna...,Die Integration der CS in die UBS macht sich –...,17.04.2024
3,Reports flow into Swiss money laundering auth...,What role does Switzerland’s financial sector ...,"Last November, in the wake of Hamas' attack on...",17.04.2024
4,"Krieg in der Ukraine, Krieg in Nahost: Doch d...",Die Welt steht in Flammen. Die Schweiz aber ma...,Die Schweizer können bekanntlich fast alles. ...,17.04.2024


#### remove duplicates

In [21]:
# Check for duplicates in the merged DataFrame, keeping all occurrences
all_occurrences_indices = nzz_data_CS_merged[nzz_data_CS_merged.duplicated(keep=False)].index

# Display the indices of all occurrences of duplicate rows
print("Indices of All Occurrences of Duplicate Rows:")
print(all_occurrences_indices)

Indices of All Occurrences of Duplicate Rows:
Index([125, 126, 571, 577, 690, 691], dtype='int64')


In [22]:
duplicate_indices_specific_columns = nzz_data_CS_merged[nzz_data_CS_merged.duplicated(subset=['article_content'], keep=False)].index
print("Indices of Duplicate Rows Based on Specific Columns:")
print(duplicate_indices_specific_columns)

Indices of Duplicate Rows Based on Specific Columns:
Index([93, 94, 96, 98, 105, 106, 125, 126, 571, 577, 690, 691], dtype='int64')


In [23]:
# Remove duplicate rows based on specific columns
nzz_data_CS_merged_no_dups = nzz_data_CS_merged.drop_duplicates(subset=['article_content'], keep='first')

In [24]:
print(len(nzz_data_CS_merged_no_dups))
print(len(nzz_data_CS_merged))

1551
1557


In [39]:
#nzz_data_CS_merged.loc[126, "article_content"]

#### Filter by date

In [25]:
# Create a copy of the DataFrames
nzz_data_CS_merged_no_dups_copy = nzz_data_CS_merged_no_dups.copy()

# Convert 'text_date' column to datetime type, considering timezone information
nzz_data_CS_merged_no_dups_copy['text_date'] = pd.to_datetime(nzz_data_CS_merged_no_dups_copy['text_date'], utc=True, format='%d.%m.%Y', errors='coerce')

# Define the start and end dates of the desired range
start_date = pd.Timestamp('2019-01-01', tz='UTC')
end_date = pd.Timestamp('2023-06-30', tz='UTC')

# Filter the DataFrame to include only articles within the specified date range
filtered_df_nzz_cs = nzz_data_CS_merged_no_dups_copy[(nzz_data_CS_merged_no_dups_copy['text_date'] >= start_date) & (nzz_data_CS_merged_no_dups_copy['text_date'] <= end_date)]

In [26]:
len(filtered_df_nzz_cs)

1239

#### Filter by relevancy

##### look for variants of Credit Suisse

In [16]:
filtered_df_nzz_cs_copy = filtered_df_nzz_cs.copy()

# Create a boolean Series where each element is True if the corresponding article contains 'credit suisse' or 'CREDIT SUISSE'

contains_lower = filtered_df_nzz_cs_copy['article_content'].str.contains('Credit-Suisse', case=True)
contains_upper = filtered_df_nzz_cs_copy['article_content'].str.contains('credit suisse', case=True)

# Combine the two Series using the bitwise OR operator

contains_either = contains_lower | contains_upper

# Check if any articles contain 'credit suisse' or 'CREDIT SUISSE'

any_contains_either = contains_either.any()

print(f"Any articles contain 'credit suisse' or 'CREDIT SUISSE'? {any_contains_either}")

num_contains_either = contains_either.sum()

print(f"Number of articles that contain 'credit suisse' or 'CREDIT SUISSE': {num_contains_either}")

Any articles contain 'credit suisse' or 'CREDIT SUISSE'? True
Number of articles that contain 'credit suisse' or 'CREDIT SUISSE': 105


##### new code

In [27]:
# Create a copy of the filtered DataFrame
filtered_df_nzz_cs_copy = filtered_df_nzz_cs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
def count_credit_suisse(text):
    return len(re.findall(r'credit[-\s]*suisse', text, re.IGNORECASE))
    
filtered_df_nzz_cs_copy['credit_suisse_count'] = filtered_df_nzz_cs_copy['article_content'].apply(count_credit_suisse)

def count_cs(text):
    return len(re.findall(r'(?<![a-zA-Z])CS(?![a-zA-Z])', text))

filtered_df_nzz_cs_copy['cs_count'] = filtered_df_nzz_cs_copy['article_content'].apply(count_cs)

# Filter articles based on multiple conditions
nzz_credit_suisse_articles = filtered_df_nzz_cs_copy[
    (filtered_df_nzz_cs_copy['credit_suisse_count'] >= 2) |
    (filtered_df_nzz_cs_copy['cs_count'] >= 2) |
    ((filtered_df_nzz_cs_copy['credit_suisse_count'] >= 1) & (filtered_df_nzz_cs_copy['cs_count'] >= 1))
]


##### old code

In [103]:
# Create a copy of the filtered DataFrame
#filtered_df_nzz_cs_copy = filtered_df_nzz_cs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
#filtered_df_nzz_cs_copy['credit_suisse_count'] = filtered_df_nzz_cs_copy['article_content'].str.count('Credit Suisse')

#def count_cs(text):
#    return len(re.findall(r'CS(?![a-zA-Z])', text))

#filtered_df_nzz_cs_copy['cs_count'] = filtered_df_nzz_cs_copy['article_content'].apply(count_cs)

# Filter articles based on multiple conditions
#nzz_credit_suisse_articles = filtered_df_nzz_cs_copy[
#    (filtered_df_nzz_cs_copy['credit_suisse_count'] >= 2) |
#    (filtered_df_nzz_cs_copy['cs_count'] >= 2) |
#    ((filtered_df_nzz_cs_copy['credit_suisse_count'] >= 1) & (filtered_df_nzz_cs_copy['cs_count'] >= 1))
#]


In [28]:
len(nzz_credit_suisse_articles)

1029

In [104]:
#len(nzz_credit_suisse_articles) #output for old method -> so 3 more articles removed.

1025

In [30]:
# Calculate the word count for each article in the credit_suisse_articles DataFrame
nzz_credit_suisse_articles.loc[:, 'word_count'] = nzz_credit_suisse_articles['article_content'].str.split().apply(len)

In [24]:
# List of article titles to exclude
#exclude_titles = [
    " Briefing am Montagmorgen",
   " Briefing am Montagabend",
    " Briefing am Dienstagmorgen",
    " Briefing am Dienstagabend",
    " Briefing am Mittwochmorgen",
    " Briefing am Mittwochabend",
    " Briefing am Donnerstagmorgen",
    " Briefing am Donnerstagabend",
    " Briefing am Freitagmorgen",
    " Briefing am Freitagabend",
    " Briefing am Wochenende",
    " Briefing"
]

# Filter the DataFrame to exclude articles with specified titles
#nzz_credit_suisse_articles_no_briefing = nzz_credit_suisse_articles[~nzz_credit_suisse_articles['article_title'].isin(exclude_titles)]

In [31]:
# Sort the articles by date in ascending order
nzz_credit_suisse_articles_sorted = nzz_credit_suisse_articles.sort_values(by='text_date')

In [32]:
# remove over 5k words
# Filter out articles with a word count higher than 5000
nzz_credit_suisse_articles_sorted = nzz_credit_suisse_articles_sorted[nzz_credit_suisse_articles_sorted['word_count'] <= 5000]

In [33]:
print(len(nzz_credit_suisse_articles_sorted))
len(nzz_credit_suisse_articles)

1027


1029

In [34]:
# Export the sorted DataFrame to a CSV file
nzz_credit_suisse_articles_sorted.to_csv('Prepped_Data/CS_NZZ_prepped.csv', index=False)

In [35]:
nzz_credit_suisse_articles_sorted.head()

,article_title,headline,article_content,text_date,credit_suisse_count,cs_count,word_count
1556,Credit Suisse stellt Zahlung von Anwaltskoste...,Die Grossbank stoppt die Zahlung der Anwaltsko...,Immer noch gibt es knapp zwei Dutzend Schweize...,2019-01-16 00:00:00+00:00,1,18,834
1555,Auf wann soll man das Wertschriftendepot wied...,Soll man Wertschriftendepots zum Jahresanfang ...,Mit dem Jahresbeginn flattern Depotauszüge und...,2019-01-28 00:00:00+00:00,1,1,1110
1554,Musical zum 200. Geburtstag: So soll der sing...,Die Macher eines Musicals zu Eschers 200. Gebu...,"Das Libretto ist geschrieben, die Songs sind k...",2019-01-29 00:00:00+00:00,2,1,608
1550,Die Schweizer Division der Credit Suisse bril...,Dank Kosteneinsparungen erreicht der Unternehm...,Banker geben viel auf Diskretion. Deshalb äuss...,2019-02-09 00:00:00+00:00,4,5,613
1549,Managerlöhne: Schweizer Grossbanken verkennen...,"UBS und Credit Suisse kämpfen, wie die meisten...","Auch dieses Jahr, spätestens nach der Veröffen...",2019-02-13 00:00:00+00:00,1,8,702


### NZZ UBS DATA

In [105]:
# import nzz cs data

file_paths = [
    "Data/NZZ/nzz_scraped_data_UBS_part1.csv",
    "Data/NZZ/nzz_scraped_data_UBS_part2.csv"
]

# List to store DataFrames
dfs = []

# Read each CSV file into a DataFrame and append to the list
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Concatenate all DataFrames into a single DataFrame
nzz_data_UBS_merged = pd.concat(dfs, ignore_index=True)

In [106]:
len(nzz_data_UBS_merged)

2593

In [107]:
nzz_data_UBS_merged.head()

,article_title,headline,article_content,text_date
0,Kultur: Wim Wenders wird Jury-Chef des Filmfe...,Die neusten Meldungen aus dem Feuilleton-Ressort.,(dpa) Der deutsche Regisseur Wim Wenders («Der...,13.06.2023
1,Lukas Gähwiler ist der neue Präsident der UBS...,Der Veteran hat es vom Kundenberater bis nach ...,Bisher hatte Lukas Gähwiler vor allem einen Jo...,12.06.2023
2,Briefing am Montagabend,"Silvio Berlusconi ist tot, die Übernahme der C...",Das Wichtigste am Abend: Silvio Berlusconi ist...,12.06.2023
3,Der Verwaltungsrat der Credit Suisse wird neu...,Die grösste Bank der Schweiz hat offiziell die...,Die Credit Suisse ist endgültig an die UBS übe...,12.06.2023
4,Briefing am Montagmorgen,"Die CS-Dekotierung, der Ölpreis, Migration übe...",Das Wichtigste am Morgen: Die endgültige Übern...,12.06.2023


In [108]:
# Check for duplicates in the merged DataFrame, keeping all occurrences
all_occurrences_indices = nzz_data_UBS_merged[nzz_data_UBS_merged.duplicated(keep=False)].index

# Display the indices of all occurrences of duplicate rows
print("Indices of All Occurrences of Duplicate Rows:")
print(all_occurrences_indices)

Indices of All Occurrences of Duplicate Rows:
Index([387, 388, 2327, 2329, 2379, 2380, 2445, 2447, 2475, 2476, 2517, 2518], dtype='int64')


In [109]:
duplicate_indices_specific_columns = nzz_data_UBS_merged[nzz_data_UBS_merged.duplicated(subset=['article_content'], keep=False)].index
print("Indices of Duplicate Rows Based on Specific Columns:")
print(duplicate_indices_specific_columns)

Indices of Duplicate Rows Based on Specific Columns:
Index([ 387,  388,  810,  930,  948, 1053, 1054, 2327, 2329, 2360, 2379, 2380,
       2445, 2447, 2475, 2476, 2484, 2517, 2518],
      dtype='int64')


In [110]:
# Remove duplicate rows based on specific columns
nzz_data_UBS_merged_no_dups = nzz_data_UBS_merged.drop_duplicates(subset=['article_content'], keep='first')

In [111]:
print(len(nzz_data_UBS_merged_no_dups))
print(len(nzz_data_UBS_merged))

2583
2593


In [81]:
#nzz_data_UBS_merged.loc[2360, "article_content"]

In [112]:
# Create a copy of the DataFrame
nzz_data_UBS_merged_no_dups_copy = nzz_data_UBS_merged_no_dups.copy()

# Convert 'text_date' column to datetime type, considering timezone information and specifying the date format
nzz_data_UBS_merged_no_dups_copy['text_date'] = pd.to_datetime(nzz_data_UBS_merged_no_dups_copy['text_date'], utc=True, format='%d.%m.%Y', errors='coerce')

# Define the start and end dates of the desired range
start_date = pd.Timestamp('2019-01-01', tz='UTC')
end_date = pd.Timestamp('2023-06-30', tz='UTC')

# Filter the DataFrame to include only articles within the specified date range
filtered_df_nzz_ubs = nzz_data_UBS_merged_no_dups_copy[(nzz_data_UBS_merged_no_dups_copy['text_date'] >= start_date) & (nzz_data_UBS_merged_no_dups_copy['text_date'] <= end_date)]


In [113]:
len(filtered_df_nzz_ubs)

2565

#### filter by relevancy

##### old code

In [49]:
# Create a copy of the filtered DataFrame
#filtered_df_nzz_ubs_copy = filtered_df_nzz_ubs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
#filtered_df_nzz_ubs_copy['UBS_count'] = filtered_df_nzz_ubs_copy['article_content'].str.count('UBS')

# Filter articles containing 'Credit Suisse' at least twice in their content
#nzz_UBS_articles = filtered_df_nzz_ubs_copy[filtered_df_nzz_ubs_copy['UBS_count'] >= 2]

##### new code

In [114]:
# Create a copy of the filtered DataFrame
filtered_df_nzz_ubs_copy = filtered_df_nzz_ubs.copy()

# Count the occurrences of the phrases 'Credit Suisse' and 'CS' in each article's content in the copied DataFrame
def count_ubs(text):
    return len(re.findall(r'(?<![a-zA-Z])UBS(?![a-zA-Z])', text))
filtered_df_nzz_ubs_copy['UBS_count'] = filtered_df_nzz_ubs_copy['article_content'].apply(count_ubs)

# Filter articles containing 'Credit Suisse' at least twice in their content
nzz_UBS_articles = filtered_df_nzz_ubs_copy[filtered_df_nzz_ubs_copy['UBS_count'] >= 2]

In [91]:
# Export the sorted DataFrame to a CSV file
#filtered_df_nzz_ubs_copy.to_csv('UBS_NZZ_unfiltered.csv', index=False)

923

In [115]:
len(nzz_UBS_articles)

923

In [117]:
# Calculate the word count for each article in the credit_suisse_articles DataFrame
nzz_UBS_articles.loc[:, 'word_count'] = nzz_UBS_articles['article_content'].str.split().apply(len)

In [66]:
# List of article titles to exclude
#exclude_titles = [
#    " Briefing am Montagmorgen",
    " Briefing am Montagabend",
    " Briefing am Dienstagmorgen",
    " Briefing am Dienstagabend",
    " Briefing am Mittwochmorgen",
    " Briefing am Mittwochabend",
    " Briefing am Donnerstagmorgen",
    " Briefing am Donnerstagabend",
    " Briefing am Freitagmorgen",
    " Briefing am Freitagabend",
    " Briefing am Wochenende",
    " Briefing"
]

# Filter the DataFrame to exclude articles with specified titles
#nzz_UBS_articles_no_briefing = nzz_UBS_articles[~nzz_UBS_articles['article_title'].isin(exclude_titles)]

In [118]:
# Sort the articles by date in ascending order
nzz_UBS_articles_sorted = nzz_UBS_articles.sort_values(by='text_date')

In [119]:
# remove over 5k words
# Filter out articles with a word count higher than 5000
nzz_UBS_articles_sorted = nzz_UBS_articles_sorted[nzz_UBS_articles_sorted['word_count'] <= 5000]

In [120]:
# Export the sorted DataFrame to a CSV file
nzz_UBS_articles_sorted.to_csv('Prepped_Data/UBS_NZZ_prepped.csv', index=False)

In [121]:
len(nzz_UBS_articles_sorted)

921